In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract

In [ ]:
!tesseract --version
!tesseract --help-psm


In [ ]:
!sudo apt-get install -y tesseract-ocr-por


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-por
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 856 kB of archives.
After this operation, 1,998 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-por all 1:4.00~git30-7274cfa-1.1 [856 kB]
Fetched 856 kB in 1s (788 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tesseract-ocr-por.
(Reading databa

In [ ]:
!tesseract --list-langs


List of available languages (3):
eng
osd
por


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Create hypothesis

In [ ]:
import os
import cv2
import pytesseract
import matplotlib.pyplot as plt

# Base folder where the subfolders with images are located
base_folder = '/content/drive/MyDrive/cropped_images'

# File path with the list of folders to be processed
image_list_file = '/content/drive/MyDrive/dataset/test_files.txt'


config = r'--oem 3 --psm 7'

# Function to perform text recognition in an image
def perform_ocr(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(img, lang="por", config=config)
    text = text.replace("\n", "").replace("\x0c", "").replace("\u2014", "")
    return text, img

with open(image_list_file, 'r') as f:
    folders_to_process = f.read().strip().split(',')
    folders_to_process = [arquivo.strip() for arquivo in folders_to_process]

for folder_name in os.listdir(base_folder):
    if folder_name in folders_to_process:
        folder_path = os.path.join(base_folder, folder_name)
        if os.path.isdir(folder_path):
            output_file = f'/content/drive/MyDrive/Tesseract/resultados/{folder_name}/hipotese.txt'
            output_folder = os.path.dirname(output_file)

            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            ocr_results = []

            for filename in os.listdir(folder_path):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(folder_path, filename)
                    ocr_text, image = perform_ocr(image_path)
                    ocr_results.append((filename.split('.')[0], ocr_text))

            with open(output_file, 'w') as txt_file:
                for image_id, ocr_text in ocr_results:
                    txt_file.write(f'{image_id} "{ocr_text}"\n')

            print(f'All results were saved in {output_file}')
